**Quantum_Randomness**



In [1]:
# install dependecncies for qiskit
!pip install qiskit
!pip install qiskit_ibm_runtime

# import dependecncies for qiskit
from qiskit import QuantumCircuit, transpile
from qiskit_ibm_runtime import Sampler, QiskitRuntimeService

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.9/363.9 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.8/75.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 9.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.3
    Uninstalling requests-2.32.3:
      Successfully uninstalled requests-2.32.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the so

In [2]:
# Authenticate using your API token
service = QiskitRuntimeService(
    token='dea47223f06d08da506a1a55df9ad66be198cfec437f8a0ca7dc2632b2cbc04a54f056488a2ce5441ba5bf38b001cc9f8308450971f0e579978c639332aa9be4',
    channel='ibm_quantum'
)

/tmp/ipython-input-2-802091477.py:2: DeprecationWarning: The "ibm_quantum" channel option is deprecated and will be sunset on 1 July. After this date, "ibm_cloud", "ibm_quantum_platform", and "local" will be the only valid channels. Open Plan users should migrate now.  All other users should review the migration guide (https://quantum.cloud.ibm.com/docs/migration-guides/classic-iqp-to-cloud-iqp)to learn when to migrate.
  service = QiskitRuntimeService(


In [3]:
# Select the least busy backend

backend = service.least_busy(operational=True, simulator=False)
print(f"Selected backend: {backend}")


Selected backend: <IBMBackend('ibm_brisbane')>


In [4]:
# Creates a quantum circuit with 128 quantum bits (qubits) and 128 classical bits for storing the measurement results.
circuit = QuantumCircuit(127, 127)
circuit.h(range(5))  # Apply Hadamard gates to all qubits
# Hadamar Gates
circuit.measure(range(127), range(127))  # Measure qubits
print("Original Circuit:")
print(circuit.draw())

Original Circuit:
       ┌───┐                                                                  »
  q_0: ┤ H ├──────────────────────────────────────────────────────────────────»
       ├───┤                                                                  »
  q_1: ┤ H ├──────────────────────────────────────────────────────────────────»
       ├───┤                                                                  »
  q_2: ┤ H ├──────────────────────────────────────────────────────────────────»
       ├───┤                                                                  »
  q_3: ┤ H ├──────────────────────────────────────────────────────────────────»
       ├───┤                                                                  »
  q_4: ┤ H ├──────────────────────────────────────────────────────────────────»
       └┬─┬┘                                                                  »
  q_5: ─┤M├───────────────────────────────────────────────────────────────────»
        └╥┘ ┌─┐       

In [5]:
# Transpile the circuit for the selected backend
# Optimizes the circuit for the chosen backend by adapting it to the hardware's gate set and constraints.

transpiled_circuit = transpile(circuit, backend=backend)
# print("Transpiled Circuit:")
# print(transpiled_circuit.draw())

In [6]:
# Define the sampler object with the backend ( allows you to submit quantum circuits for execution and get results)
sampler = Sampler(mode=backend)

In [7]:
# Run the transpiled circuit
job = sampler.run([transpiled_circuit])
result = job.result()

/usr/local/lib/python3.11/dist-packages/qiskit_ibm_runtime/qiskit_runtime_service.py:1129: UserWarning: IBM Quantum Platform is moving, and this version will be sunset on July 1. To get started on the new platform, read the migration guide at https://docs.quantum.ibm.com/migration-guides/classic-iqp-to-cloud-iqp
  warnings.warn(warning_message)


In [8]:
#  Collapse During Measurement: When the qubits are measured, their states collapse due to decoherence.
#  Each qubit independently resolves into producing a binary value. For example, a possible measurement outcome for 5 qubits might be 10110.

dist = result[0].data.c.get_counts().keys()
binary = list(dist)[0]
decimal = int(binary, 2)
print('Binary', binary, 'Decimal', decimal)

Binary 0000000000000000000000100000000000000000100100000000000010000000000000000000000000000000000000000000000000000000000000000111111 Decimal 20282496647491274297965241434175


In [13]:
# we could try the same step for a number of iterations
iterations = 3
for i in range(iterations):
    # Run and get counts

    result = sampler.run([transpiled_circuit]).result()
    counts = result[0].data.c.get_counts()

    # Get the single key (bitstring) from counts
    quantum_bin = list(counts.keys())[0]

    # Convert to integer
    quantum_num = int(quantum_bin, 2)

    # Convert to various formats
    quantum_dec = quantum_num
    quantum_hex = hex(quantum_num)[2:]
    quantum_oct = oct(quantum_num)[2:]

    print(f"Quantum RNG Results for iteration {i+1}:")
    print("Binary:      ", quantum_bin)
    print("Decimal:     ", quantum_dec)
    print("Hexadecimal: ", quantum_hex)
    print("Octal:       ", quantum_oct)
    print('')

Quantum RNG Results for iteration 1:
Binary:       0000000000001000000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000100000000101000
Decimal:      20769187434139310514684935270318120
Hexadecimal:  40000000000000002000000004028
Octal:        100000000000000000000020000000000040050

Quantum RNG Results for iteration 2:
Binary:       0000000000000000000000010000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000010111
Decimal:      10141204801825835211973625643031
Hexadecimal:  80000000000000000000000017
Octal:        20000000000000000000000000000000027

Quantum RNG Results for iteration 3:
Binary:       0000100000000000000000000000000000000000000000000000000000000000000000001000000000000000001000000000000000000000000000000011110
Decimal:      5316911983139663491633242708350337054
Hexadecimal:  400000000000000004000100000001e
Octal:        40000000000000000000001000001000000000036

